In [1]:
import requests
import json
import pandas as pd
import numpy as np
import pymysql
import os

In [3]:
# 1. 영화id를 사용하기 위해 영화정보 파일가져오기 
tmdb_movie_list = pd.read_csv('../data-files/total_tmdbmovielist.csv', index_col=0) 

C:\Users\user\.conda\envs\ykseo\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,4,7,12,17,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# 2. 영화정보 중 id만 가져오기
tmdb_movie_list.info()
tmdb_movie_id_list = tmdb_movie_list[["id"]]
# del tmdb_movie_list

<class 'pandas.core.frame.DataFrame'>
Int64Index: 630159 entries, 0 to 630185
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   adult                  630158 non-null  object 
 1   backdrop_path          150102 non-null  object 
 2   belongs_to_collection  11995 non-null   object 
 3   budget                 630157 non-null  object 
 4   genres                 630155 non-null  object 
 5   homepage               69801 non-null   object 
 6   id                     630156 non-null  object 
 7   imdb_id                396414 non-null  object 
 8   original_language      630157 non-null  object 
 9   original_title         630130 non-null  object 
 10  overview               25238 non-null   object 
 11  popularity             630119 non-null  object 
 12  poster_path            426255 non-null  object 
 13  production_companies   630119 non-null  object 
 14  production_countries   630088 non-nu

In [5]:
print(tmdb_movie_id_list.shape)
tmdb_movie_id_list.head()

(630159, 1)


,id
0,2.0
1,3.0
2,5.0
3,6.0
4,8.0


In [9]:
## 영화id를 이용하여 crew정보 crawling  (너무 오래걸림)

all_crew_list = []
all_cast_list = []
error_list = []
for idx,  movie_id in enumerate(tmdb_movie_id_list["id"].values):
    try :
        crew_url="https://api.themoviedb.org/3/movie/{0}?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"
        response = requests.get(crew_url.format(movie_id))
        json_object = response.content
        movie_data = json.loads(json_object)
        
        # crew_list += movie_data["credit"]['crew']
        crew_list = movie_data["credits"]["crew"]
        for crew in crew_list:
            crew["movie_id"] = movie_id
            all_crew_list.append(crew)
    except:
        error_list.append(movie_id)

    if idx == 500:
        break

In [2]:
# 현재 상영 중인 영화
now_playing_movies=pd.read_csv("../data-files/now_playing_movies.csv", index_col=0)

In [3]:
now_playing_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              200 non-null    bool   
 1   backdrop_path      200 non-null    object 
 2   genre_ids          200 non-null    object 
 3   id                 200 non-null    int64  
 4   original_language  200 non-null    object 
 5   original_title     200 non-null    object 
 6   overview           160 non-null    object 
 7   popularity         200 non-null    float64
 8   poster_path        200 non-null    object 
 9   release_date       200 non-null    object 
 10  title              200 non-null    object 
 11  video              200 non-null    bool   
 12  vote_average       200 non-null    float64
 13  vote_count         200 non-null    int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 20.7+ KB


In [4]:
now_playing_movies_id_list = now_playing_movies[["id"]]
now_playing_movies_id_list

,id
0,580489
1,512195
2,585245
3,566525
4,568124
...,...
195,116322
196,817566
197,8373
198,258489


In [10]:
all_crew_list = []
error_list = []
for idx,  movie_id in enumerate(now_playing_movies_id_list["id"].values):
    try :
        crew_url="https://api.themoviedb.org/3/movie/{0}?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"
        response = requests.get(crew_url.format(movie_id))
        json_object = response.content
        movie_data = json.loads(json_object)
        
        # crew_list += movie_data["credit"]['crew']
        crew_list = movie_data["credits"]["crew"]
        for crew in crew_list:
            crew["movie_id"] = movie_id
            all_crew_list.append(crew)
    except:
        error_list.append(movie_id)

    #if idx == 500:
        #break

In [11]:
# crew 정보 csv파일로 저장 
crews=pd.DataFrame(all_crew_list)
crews.head(2)


,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,credit_id,department,job,movie_id
0,False,2,149,Camera,Robert Richardson,Robert Richardson,1.842,/lmJy31oZtN5l2VAGpkrt3xEXsxL.jpg,5d59628f61bac40eca4a728e,Camera,Director of Photography,580489
1,False,2,1333,Acting,Andy Serkis,Andy Serkis,10.922,/2aJLwOQK50Lx7PvIHGW1GMytTOL.jpg,5e866a52a5743d00180fe64e,Directing,Director,580489


In [12]:
crews.to_csv("../data-files/crew.csv", header=True, index=True)

In [14]:
# crew정보 DB(mysql)에 저장 
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")

cursor = conn.cursor()
cursor.execute('DELETE FROM crew')
#error_list=[]
for a in crews.values:
    print(a)
    #try:
    sql= "INSERT INTO crew (adult, gender, id, known_for_department, name,original_name, popularity, profile_path,credit_id, department,job,movie_id) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, list(np.array(a)))
    #except:
        ##error_list.append(a)


conn.commit() # 이전에 실행된 SQL 결과를 확정
cursor.close()
conn.close()

[False 2 149 'Camera' 'Robert Richardson' 'Robert Richardson' 1.842
 '/lmJy31oZtN5l2VAGpkrt3xEXsxL.jpg' '5d59628f61bac40eca4a728e' 'Camera'
 'Director of Photography' 580489]
[False 2 1333 'Acting' 'Andy Serkis' 'Andy Serkis' 10.922
 '/2aJLwOQK50Lx7PvIHGW1GMytTOL.jpg' '5e866a52a5743d00180fe64e' 'Directing'
 'Director' 580489]
[False 2 2524 'Acting' 'Tom Hardy' 'Tom Hardy' 44.119
 '/9bXUMLOBCcnhcUGtfw0pdqbvpiH.jpg' '6099c48f3520e8003f011485'
 'Production' 'Producer' 580489]
[False 2 2524 'Acting' 'Tom Hardy' 'Tom Hardy' 44.119
 '/9bXUMLOBCcnhcUGtfw0pdqbvpiH.jpg' '619fdfc04f9a99002458996d' 'Writing'
 'Story' 580489]
[False 2 7229 'Sound' 'Marco Beltrami' 'Marco Beltrami' 2.303
 '/fb3TfFITlOC0BN3kNpUXj1FL0LN.jpg' '5fdaa611eda4b700403fbc88' 'Sound'
 'Original Music Composer' 580489]
[False 0 5545 'Production' 'K.C. Hodenfield' 'K.C. Hodenfield' 3.482 None
 '6157e0e01c635b00447c7d13' 'Production' 'Co-Producer' 580489]
[False 0 5545 'Production' 'K.C. Hodenfield' 'K.C. Hodenfield' 3.482 None